## Presented by Baha Altan Reisoğlu

In [83]:
!pip install --upgrade pip
!pip -q  install --upgrade transformers peft bitsandbytes
!pip -q  install trl  
!pip -q install langchain
!pip install -U langchain-community
!pip install jq
!pip install sentence-transformers
!pip install chromadb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.5/605.5 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 124.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 92.4 MB/s eta 0:00:00
  Created wheel for pypika: filename=PyPika-0.48.9-py2.py3-none-any.whl size=53725 sha256=3672a3ad30b1aa2d2c7c691da2ec9ff981d4ba00b307c9768454bedf802c011d
  Stored in directory: /root/.cache/pip/wheels/e1/26/51/d0bffb3d2fd82256676d7ad3003faea3bd6dddc9577af665f4
Successfully built pypika
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependenc

In [205]:
from transformers import AutoTokenizer, AutoModelForCausalLM,BitsAndBytesConfig
import torch
import numpy as np
import pandas as pd

import bitsandbytes as bnb
import torch.nn as nn
from chromadb.utils import embedding_functions
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.text_splitter import RecursiveCharacterTextSplitter,SentenceTransformersTokenTextSplitter
from sentence_transformers import SentenceTransformer
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from langchain.document_loaders import JSONLoader
from langchain.vectorstores import Chroma
from langchain.schema import Document

In [3]:
bnbConfig = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float32,  # Daha geniş aralık
    
)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device.type

'cuda'

In [51]:
#some parameters or things that will be used 
class Config():
    model_id = "/kaggle/input/gemma-2/transformers/gemma-2-2b-it/2/"
    tensorflow_batch=32
    token_limit = 512
    num_data_limit = 1000
    lora_name = "instructional"
    lora_rank = 16
    lora_alpha=32
    lr_value = 1e-4 #best learning_rate
    train_epoch = 7
    max_steps = 100
    hf_data_path = "merve/turkish_instructions"#input and output about daily topics
    weight_decay=0.01,
    epsilon=1e-6
    adapter_name="instructional"
    device_map="auto"
    epoch=15
    adam_beta1 = 0.9
    adam_beta2 = 0.995
    adam_epsilon = 1e-8
    max_grad_norm = 1.0
    top_k=50
    top_p=0.9
    tempe=1.0
    
    

In [6]:
if device.type =="cuda":
    model = AutoModelForCausalLM.from_pretrained(
        Config.model_id,
        quantization_config=bnbConfig,
        device_map= "auto",
        trust_remote_code=True,  torch_dtype=torch.float32
        )
    
    print("& cuda")
else:
    model = AutoModelForCausalLM.from_pretrained(
        Config.model_id,
        
        device_map="auto",
        trust_remote_code=True,
        )
    print("not cuda")

tokenizer = AutoTokenizer.from_pretrained(Config.model_id)
eos_token = tokenizer.eos_token
pad_token = tokenizer.pad_token
tokenizer.padding_side = "right"


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

& cuda


In [7]:
print("train_adapter" in dir(model))
print(model.dtype)

False
torch.float32


In [52]:

def text_generator(prompt):
    input_text = f"<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n"
    input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
    
    outputs = model.generate(**input_ids, max_new_tokens=Config.token_limit,do_sample=True,
                          temperature=Config.tempe,
                          top_p=Config.top_p,
                          top_k=Config.top_k,
                          repetition_penalty=1.0)
    print(tokenizer.decode(outputs[0],skip_special_tokens=True))

# Örnek kullanım


In [9]:
text_generator("Fransız tostu için malzemeler nedir")

The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


user
Fransız tostu için malzemeler nedir
model
## Fransız Toastu Malzemeleri:

**Temel Malzemeler:**

* **Tostu Ekmek:**  Sıcak bir fırın,  ya da evinizde kolayca hazırlayabileceğiniz  **yüzük ekmek** (bunlar genellikle Fransız toast'u için kullanılıyor). 
* **Yağ veya Tereyağı:**  Tıpkı ekmeği kızartmak için kullandığınız yağ, daha koyu bir tat ve daha yumuşak bir tat elde etmenize yardımcı olabilir. 
* **Bir Yağlı**  (bir kase)  : Yağ veya tereyağı ile birlikte kullanıldığında daha iyi bir sonuç elde etmek için kullanılır.
* **Ayrı bir kasede**  :  Üzüntü, tatlandırma veya diğer malzemeler için kullanılabilir. 
* **Sirke:**  Bu, Fransız toast'u için bir tatlandırıcı olarak kullanılır.

**Alternatif Malzemeler:**

* **Domates:**   Ekmek için. 
* **Salatalık:**   Ekmek için.
* **Mercimek:**  Ekmek için. 
* **Mayonez:**  Ekmek için. 
* **İsteğe bağlı eklemeler:**   Yağ, tereyağı, sirke, limon suyu, tuz, karabiber, kekik, biber, kimyon, patates, domates,  fındık,  ve diğer malzemeler ekl

In [10]:
text_generator("Gülmenin neden önemli olduğunu açıklayabilir misiniz?")

user
Gülmenin neden önemli olduğunu açıklayabilir misiniz?
model
Gülmek, sadece gülmenin tadını çıkarmak gibi basit bir durumdan ibaret değil, aslında beden ve ruh sağlığımızı önemli ölçüde etkileyen bir olaydır. 

**Gülmenin faydaları şunlardır:**

**Psikolojik Faydalar:**

* **Stres ve depresyonu azaltır:** Gülmek, stres hormonu olan kortizol seviyelerini düşürerek, rahatlama ve sakinleşme hissine neden olur. 
* **Zihninizi dinlendirir:** Gülmek, zihninizi dinlendirmek ve düşüncelerinizin yoğunlaşmasına izin vererek, sakin bir ruh hali sağlar. 
* **İç huzur ve mutluluk hissini artırır:** Gülmek, hem fiziksel hem de zihinsel olarak mutlu hissetmenize yardımcı olur. 
* **Kendinizi daha iyi hissedersiniz:**  Gülmek, kendiliğinden pozitif duygulara ve kendinize olan saygına neden olur. 
* **Kişiliğinizi daha pozitif gösterir:** Gülmek, diğer insanlara daha pozitif ve sıcak bir insan gibi hissetmenize yardımcı olur.

**Fiziksel Faydalar:**

* **Tıbbi faydalar:** Gülmek, vücudun rahatlama 

In [11]:
from datasets import load_dataset

# Türkçe bir örnek: "turkic-xwmt,turkish_instructions" veri kümesi
dataset = load_dataset(Config.hf_data_path)

train_data = dataset["train"]
train_df = pd.DataFrame(train_data)
train_df

def merge_instruct_input(rows):
    if(rows[" giriş"]!=None):
        rows["talimat"]=rows["talimat"]+"\n"+rows[" giriş"]
    return rows

merged_train=train_df.apply(merge_instruct_input,axis=1).sample(frac=0.3).drop(" giriş",axis=1)

README.md:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

instructions.csv:   0%|          | 0.00/21.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51563 [00:00<?, ? examples/s]

In [12]:
model.dtype

torch.float32

In [13]:
merged_train.drop("Unnamed: 0",inplace=True,axis=1)

In [14]:
full=[]
for i in range(len(merged_train)):
    full.append(f"<start_of_turn>user\n{merged_train['talimat'].iloc[i]}<end_of_turn>\n<start_of_turn>model\n{merged_train[' çıktı'].iloc[i]}<end_of_turn>"+eos_token)

In [15]:
model.dtype

torch.float32

In [16]:
merged_train["full Context"]=np.array(full)

merged_train


talimat  \
18541   iPhone ve Android telefon karşılaştırması yaz...   
10521  Aşağıdaki cümleyi daha kısa olacak şekilde yen...   
43093   Nostalji duygusu taşıyan bir şarkı sözü oluşt...   
23531   Öğelerin bir listesi verildiğinde, ilginç bir...   
31363   Aşağıdaki kodu değerlendirin ve hataları bild...   
...                                                  ...   
33963   Çevrimiçi satışların büyümesine dayalı olarak...   
18454   Size bir cümle sunulur. Cümleyi 3. şahıs bakı...   
26738   Kullanıcı verilerini toplamak için üç En İyi ...   
43814   Bir yazılım güncellemesi için bir değişiklik ...   
1960    Pandemi sırasında kullanılan veri gözetiminin...   

                                                   çıktı  \
18541  IPhone'lar şık tasarımı ve yazılım entegrasyon...   
10521   Okuduğum yazı oldukça ilgi çekici ve eğiticiydi.   
43093  "Ne zaman bu kadar kaygısız güleceğimizi hatır...   
23531   Neden bir çizim yarışması denemiyorsunuz? Her...   
31363   5. satırdaki kodda bir hata var. "selamlama" ...   
...                                                  ...   
33963   Yeni bir iş fikri, işletmeler ve bireyler içi...   
18454                                  O mağazaya gitti.   
26738   Kullanıcı verilerini toplamak için Üç En İyi ...   
43814   -Hata düzeltmeleri: uygulamanın çökmesine ned...   
1960   Pandemi sırasında uygulanan veri gözetiminin s...   

                                            full Context  
18541  <start_of_turn>user\n iPhone ve Android telefo...  
10521  <start_of_turn>user\nAşağıdaki cümleyi daha kı...  
43093  <start_of_turn>user\n Nostalji duygusu taşıyan...  
23531  <start_of_turn>user\n Öğelerin bir listesi ver...  
31363  <start_of_turn>user\n Aşağıdaki kodu değerlend...  
...                                                  ...  
33963  <start_of_turn>user\n Çevrimiçi satışların büy...  
18454  <start_of_turn>user\n Size bir cümle sunulur. ...  
26738  <start_of_turn>user\n Kullanıcı verilerini top...  
43814  <start_of_turn>user\n Bir yazılım güncellemesi...  
1960   <start_of_turn>user\n Pandemi sırasında kullan...  

[15469 rows x 3 columns]

In [17]:
merged_train["talimat"].loc[49029]

' Aşağıdaki bilgilere göre bir bilmece oluşturunuz. Bilmecenin çıktısını alın.\n Suda yaşayan küçük bir yaratık'

In [18]:
text_generator("Aşağıdaki bilgilere göre bir bilmece oluşturunuz. Bilmecenin çıktısını alın.\n Suda yaşayan küçük bir yaratık")

user
Aşağıdaki bilgilere göre bir bilmece oluşturunuz. Bilmecenin çıktısını alın.
 Suda yaşayan küçük bir yaratık
model
Bilmece: 

**Düşünür, ancak konuşmaz, 
İçinde fırtınalar, ama taşır hiçbir şey, 
Suda kayar, ama taşmaz, 
Güneşin altında, gece de parlar.**

**Çıktı:** 
**Deniz faresi** 

**Açıklama:** 

Deniz faresi, suda yaşayan küçük bir yaratık. Bilmecedeki detaylar, deniz faresi hakkında bilgi verir. 

* **Düşünür, ancak konuşmaz:** Deniz faresi, insan gibi konuşmaz, ancak düşünür. 
* **İçinde fırtınalar, ama taşır hiçbir şey:** Deniz faresi, suda fırtına gibi dalgalanır, ancak taşır hiçbir şey. 
* **Suda kayar, ama taşmaz:** Deniz faresi, suda kayar, ancak taşmaz.
* **Güneşin altında, gece de parlar:** Deniz faresi, güneş altında ve gece de parıldar. 


 
 



In [19]:
merged_train.iloc[4]["talimat"]

" Aşağıdaki kodu değerlendirin ve hataları bildirin.\n def salutation(isim): if name == 'bob': selamlama = 'Merhaba' else: selamlama = 'Hoşçakal' print(selamlama)"

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
from datasets import load_dataset,DatasetDict,Dataset
train_dataset = Dataset.from_pandas(merged_train)

train_dataset = train_dataset.remove_columns(["__index_level_0__"])
train_dataset

Dataset({
    features: ['talimat', ' çıktı', 'full Context'],
    num_rows: 15469
})

In [22]:
def tokenize_function(examples):
    tokenized = tokenizer(
        examples["full Context"],
        padding="max_length",
        truncation=True,
        max_length=Config.token_limit,
        return_tensors="pt"
    ).to("cuda")
    # Labels are identical to input_ids for causal language modeling
    tokenized["labels"] = tokenized["input_ids"].clone()
    return tokenized
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["full Context"])
print("Dataset tokenized:", train_dataset[0])

Map:   0%|          | 0/15469 [00:00<?, ? examples/s]

Dataset tokenized: {'talimat': ' iPhone ve Android telefon karşılaştırması yazınız.', ' çıktı': "IPhone'lar şık tasarımı ve yazılım entegrasyonuyla tanınırken, Android'ler özelleştirilebilirlikleri ve cihazı modlama ve rootlama yetenekleriyle tanınır. IPhone donanımı daha pahalı olma eğilimindeyken, Android telefonlar istediğiniz özelliklere bağlı olarak bir dizi fiyat sunar. IPhone kullanıcıları Apple App Store'a, Android kullanıcıları ise Google Play Store'a ve Android'e özel diğer uygulama mağazalarına erişebilir. Ek olarak, iPhone kullanıcılarının medya yönetimi için iTunes'u kullanmaları gerekirken, Android kullanıcılarının PC veya bulut hizmetlerini kullanmaları gerekir.", 'input_ids': [2, 106, 1645, 108, 13200, 1408, 9989, 27999, 234587, 50035, 1126, 25725, 38154, 34806, 235265, 107, 108, 106, 2516, 108, 2520, 126496, 235303, 2596, 7736, 10622, 79527, 34544, 1408, 176134, 235262, 659, 110407, 2715, 477, 215085, 4682, 168825, 1753, 235269, 9989, 235303, 1595, 97256, 3510, 36898, 

In [23]:
model.dtype

torch.float32

In [24]:
train_dataset

Dataset({
    features: ['talimat', ' çıktı', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 15469
})

In [25]:
"""from typing import Dict
text=[]


# Iterate over the rows in the filtered DataFrame
for i, row in merged_train.iterrows():
    prompt = row['talimat']  # Question
    output = row[' çıktı']  # Answer

    # Construct the conversation
    conversation = (
        f"<start_of_turn>user\n{prompt}<end_of_turn>\n"
        f"<start_of_turn>model\n{output}<end_of_turn>"
    )

    # Tokenize and check the length
    tokenized=tokenizer(conversation)
    
    # Skip data if the token length is longer than our limit
    if len(tokenized["input_ids"]) < Config.token_limit:
        text.append(tokenized)
        if len(text) >= Config.num_data_limit:
            break

print(f"Number of training samples: {len(text)}")
print("\nSample conversation:")
print(text)"""

'from typing import Dict\ntext=[]\n\n\n# Iterate over the rows in the filtered DataFrame\nfor i, row in merged_train.iterrows():\n    prompt = row[\'talimat\']  # Question\n    output = row[\' çıktı\']  # Answer\n\n    # Construct the conversation\n    conversation = (\n        f"<start_of_turn>user\n{prompt}<end_of_turn>\n"\n        f"<start_of_turn>model\n{output}<end_of_turn>"\n    )\n\n    # Tokenize and check the length\n    tokenized=tokenizer(conversation)\n    \n    # Skip data if the token length is longer than our limit\n    if len(tokenized["input_ids"]) < Config.token_limit:\n        text.append(tokenized)\n        if len(text) >= Config.num_data_limit:\n            break\n\nprint(f"Number of training samples: {len(text)}")\nprint("\nSample conversation:")\nprint(text)'

In [26]:
print("Available modules in the Gemma model:")
for name, module in model.named_modules():
    print(name)

Available modules in the Gemma model:

model
model.embed_tokens
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.self_attn.rotary_emb
model.layers.0.mlp
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.0.mlp.act_fn
model.layers.0.input_layernorm
model.layers.0.post_attention_layernorm
model.layers.0.pre_feedforward_layernorm
model.layers.0.post_feedforward_layernorm
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.self_attn.rotary_emb
model.layers.1.mlp
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.1.mlp.act_fn
model.layers.1.input_layernorm
model.layers.1.post_attention_layernorm
model.layers.1.pre_feedforward

# Adapter Or Fine Tuning 

### i decided to use fine-tune but can also add RAG and Adapter Tuning this function

In [27]:
"""#freezing the original weight of model
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)"""

'#freezing the original weight of model\nfor param in model.parameters():\n  param.requires_grad = False  # freeze the model - train adapters later\n  if param.ndim == 1:\n    # cast the small parameters (e.g. layernorm) to fp32 for stability\n    param.data = param.data.to(torch.float32)\n\nmodel.gradient_checkpointing_enable()  # reduce number of stored activations\nmodel.enable_input_require_grads()\n\nclass CastOutputToFloat(nn.Sequential):\n  def forward(self, x): return super().forward(x).to(torch.float32)\nmodel.lm_head = CastOutputToFloat(model.lm_head)'

In [28]:
from peft import LoraConfig, get_peft_model, PeftModel
lora_config = LoraConfig(
    r=64,  
    lora_alpha=32,  
    target_modules=["q_proj", "v_proj"],  #basic modeules
    task_type="CAUSAL_LM",
)


In [29]:
model.dtype

torch.float32

In [30]:
"fit" in dir(model)

False

In [31]:
#how many parameter can be trained
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

print_trainable_parameters(model)

trainable params: 590065920 || all params: 1602203904 || trainable%: 36.82839110096189


In [32]:
model.dtype

torch.float32

In [33]:
import transformers

In [34]:
for param in model.parameters():
    print(param.requires_grad)

True
False
False
False
False
False
False
False
True
True
True
True
False
False
False
False
False
False
False
True
True
True
True
False
False
False
False
False
False
False
True
True
True
True
False
False
False
False
False
False
False
True
True
True
True
False
False
False
False
False
False
False
True
True
True
True
False
False
False
False
False
False
False
True
True
True
True
False
False
False
False
False
False
False
True
True
True
True
False
False
False
False
False
False
False
True
True
True
True
False
False
False
False
False
False
False
True
True
True
True
False
False
False
False
False
False
False
True
True
True
True
False
False
False
False
False
False
False
True
True
True
True
False
False
False
False
False
False
False
True
True
True
True
False
False
False
False
False
False
False
True
True
True
True
False
False
False
False
False
False
False
True
True
True
True
False
False
False
False
False
False
False
True
True
True
True
False
False
False
False
False
False
False
True
True
True
True
Fal

In [35]:
train_dataset

Dataset({
    features: ['talimat', ' çıktı', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 15469
})

In [36]:
model.to(device)
from transformers import Trainer, TrainingArguments
from trl import SFTTrainer
from transformers import DataCollatorForSeq2Seq

train_args = TrainingArguments(
    per_device_train_batch_size=1, 
    gradient_accumulation_steps=8,  
    warmup_steps=100,
    max_steps=600,
    num_train_epochs=7,  
    gradient_checkpointing=True,
      # 16-bit float
    logging_steps=500,
    optim="adamw_torch",  
    weight_decay=0.01,
    lr_scheduler_type="linear",
    output_dir="outputs",
    save_steps=500,
    save_total_limit=2,
    logging_dir="logs",  
    report_to="none",
)


data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding="longest",
    return_tensors="pt"#i used SeqtoSeq because all datas have to got same length
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=train_args,
    peft_config=lora_config,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

trainer.train()

                      



print(model.dtype)

<ipython-input-36-4e24c476a087>:33: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_c

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


torch.float32


In [46]:
model.save_pretrained("/kaggle/working/gemma-finetuned-altan")

In [38]:
print("OKAY")

OKAY


In [54]:
text_generator("Kayıp bir cep telefonunu bulmak için kullanılan işlemi açıklayın")

user
Kayıp bir cep telefonunu bulmak için kullanılan işlemi açıklayın
model
Kayıp bir cep telefonunda birleştirme işlemi, telefonun kaybolduğu yerden birleştirme, yakalanması veya bulunması için birleştirilmesi veya geri alma işlemini içeren bir süreçtir. İki farklı cihaz arasında cihazı bulaştırmak için kullanılabilir.


In [53]:
text_generator("iPhone ve Android telefon karşılaştırması yazınız.")

user
iPhone ve Android telefon karşılaştırması yazınız.
model
iPhone ve Android, 2010'lardan beri telefon dünyasına baskı yapmaktan sorumludur. Öngörülemeyen bir hızla genişledikleri bir pazar olarak, birbiriyle rekabet etmek zorunda kalan iki platform arasında belirgin farklılıklar vardır. Apple, birleşik platformu iPhone'u, iOS'u ve işbirliği platformu, Apple Watch'i sunar. Google'un Android, Android'yi ve işbirliği platformu, Google Pixel'i sunar.


# RAG Released .Then ADAPTER TUNING is COMING SOON

In [119]:
import os
import json

# JSON dosya yolunu ayarla
file_path = os.path.join(os.getcwd(), "/kaggle/input/turkish-dataset-1/squad-tr-dev-v1.0.0-excluded.json")

# JSON dosyasını oku
with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

print(data)

{'version': 'v2.0', 'data': [{'title': 'Normanlar', 'paragraphs': [{'context': "Rollo'nun Vikinglerinin torunları ve Frenk eşleri İskandinav dinini ve Eski İskandinav dilini Katoliklik (Hıristiyanlık) ve yerel halkın Gallo-Romance dilinin yerini alacaktı, annelik Frenk mirasını Eski İskandinav gelenekleri ve gelenekleriyle benzersiz bir “Norman” sentezlemek için Fransa'nın kuzeyinde kültür. Norman dili, Norveççe konuşan bir iktidar sınıfı tarafından Romance yerli langue d'oïl şubesinin benimsenmesi ile dövüldü ve bugün hayatta kalan bölgesel dil haline geldi.\n", 'qas': [{'id': '56dde27d9a695914005b9651', 'question': 'Norman dini neydi?', 'answers': [{'text': 'katoliklik'}, {'text': 'katoliklik', 'answer_start': 94}, {'text': 'katoliklik', 'answer_start': 94}]}]}, {'context': "Normanlar bundan sonra Fransa'nın geri kalanının feodal doktrinlerini benimsediler ve Normandiya ve İngiltere'de işlevsel bir hiyerarşik sistem haline getirdiler. Yeni Norman hükümdarları kültürel ve etnik açıdan

In [206]:
class PREPARE_CHROMA():
    def __init__(self,file_path):
        self.file_path=file_path
        
        
    def pre(self):
        loader = JSONLoader(
            file_path=self.file_path,
            jq_schema='''
            .data[] | .paragraphs[] | {
                content: .context, 
                metadata: {
                    questions: [.qas[].question], 
                    answers: [.qas[].answers[].text]
                }
            }
            ''',
            text_content=False  # page_content olmayan veri için gerekli
        )
        documents = loader.load()
        return documents
        
    def metadata(self):
        documents=self.pre()
        for doc in documents:
            for key, value in doc.metadata.items():
                if isinstance(value, list):
                    # Listeyi düz metin yap, virgülle ayırarak birleştir
                    doc.metadata[key] = ', '.join(map(str, value))
        return documents
    


    def __call__(self):
       processed_documents=self.metadata()
       embedding_model = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
       chroma_store = Chroma.from_documents(processed_documents, embedding_model, persist_directory="chroma_db_altan")
       return chroma_store 
   

In [207]:
mein_chroma=PREPARE_CHROMA("/kaggle/input/turkish-dataset-1/squad-tr-dev-v1.0.0-excluded.json")()

In [209]:
query_text = "Edward kimi Canterbury başpiskoposu yaptı?"
results = mein_chroma.similarity_search(query_text, k=5)

results

[Document(metadata={'answers': 'Jumièges (Robert), Jumièges (Robert), Jumièges (Robert)', 'questions': 'Edward kimi Canterbury başpiskoposu yaptı?'}, page_content="Sonunda Confessor Edward 1041'de babasının sığınağından döndüğünde üvey kardeşi Harthacnut'un daveti üzerine yanında Norman eğitimli bir zihin getirdi. Ayrıca bazıları İngiliz süvari kuvveti kuran birçok Norman danışman ve savaşçı getirdi. Bu kavram gerçekten kök salmadı, ancak Edward'ın tutumlarının tipik bir örneğidir. Canterbury başpiskoposu Jumièges'den Robert'ı atadı ve Ralph'i Hereford'un ürkek kontu yaptı. Kayınbiraderi II. Eustace, Boulogne Kontu 1051'de mahkemesine davet etti. Bu olay Sakson ve Norman arasındaki en büyük erken çatışmalara neden oldu ve sonunda Wessex Kontu Godwin'in sürgünüyle sonuçlandı.\n"),
 Document(metadata={'answers': 'Dokumacılar, Dokumacılar, Dokumacılar, bir restoran, restoran, bir restoran', 'questions': "Canterbury'de bir dokuma okulunun olduğu ev hanesiydi?, Şu anda Dokuma Evi sitesinde 

In [214]:
class Text_Generator():
    def __init__(self):
        pass
        
    def text_generator_query(self,query):
        role_description = """You are a helpful expert assistant. 
                           Provide an example answer to the given question, that might be found in a document like an encyclopedia,book  etc."""
    
        input_text = (
            f"{role_description}\n"
            f"<start_of_turn>user\n{query}<end_of_turn>\n<start_of_turn>model\n"
        )
        input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
        
        outputs = model.generate(**input_ids, max_new_tokens=Config.token_limit)
        response=tokenizer.decode(outputs[0])
        joint_query = f"{query} {response}"
        results = chroma_store.similarity_search(query_text, k=2)
        retrieved_documents = [results[i].metadata["answers"][0] for i in range(len(results))]
        return retrieved_documents
        
    def __call__(self,query):
        relevant_chunks=self.text_generator_query(query)
        context = "\n\n".join(relevant_chunks)
        role_description = (f"""You are an assistant for question-answering tasks. Use the following pieces of
                retrieved context to answer the question. If you don't know the answer, say that you 
                don't know.
                \n\nContext:\n" + {context} + "\n\nQuestion:\n" + {query}"""
        )
        input_text = (
            f"{role_description}\n"
            f"<start_of_turn>user\n{query}<end_of_turn>\n<start_of_turn>model\n"
        )
        input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
        
        outputs = model.generate(**input_ids, max_new_tokens=Config.token_limit,do_sample=True,
                              temperature=Config.tempe,
                              top_p=Config.top_p,
                              top_k=Config.top_k,
                              repetition_penalty=1.0)
        result=tokenizer.decode(outputs[0],skip_special_tokens=True)
        return result


In [215]:
generator=Text_Generator()
result=generator("iPhone ve Android telefon karşılaştırması yazınız.")

In [216]:
result

'You are an assistant for question-answering tasks. Use the following pieces of\n                retrieved context to answer the question. If you don\'t know the answer, say that you \n                don\'t know.\n                \n\nContext:\n" + J\n\nD + "\n\nQuestion:\n" + iPhone ve Android telefon karşılaştırması yazınız.\nuser\niPhone ve Android telefon karşılaştırması yazınız.\nmodel\n iPhone ve Android telefon karşılaştırması şunlar hakkında ayrıntılı bir analizdir: Apple iPhone, Apple tarafından üretilen, iOS işletim sistemi ile çalışan, Apple\'ın kendi yazılımı olan iOS\'dan oluşan bir Apple iPhone serisiidir. iPhone, Apple\'ın kendi tasarladığı ve geliştirip, yazılımı olan Apple\'ın kendi yazılımı olan iOS\'dan oluşan bir Apple iPhone serisiidir. Android telefonlar, Google tarafından üretilen, Android işletim sistemini ve Google tarafından geliştirilmiş yazılımıyla çalışan, Android\'in kendi yazılımı olan Android işletim sisteminden oluşan bir Android telefon serisiidir.'

In [ ]:
print("OKAAAY FINISHED (for now)")